<a href="https://colab.research.google.com/github/aashu1328/Natural-language-Processing/blob/master/NLP_Class6_accuracy_countvector_cleandata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy
import nltk
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer,TfidfVectorizer
from nltk import word_tokenize
import string 
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts,GridSearchCV
from sklearn.metrics import classification_report

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [0]:
def stop_tokenize(Sentence):
  word_tokens = word_tokenize(remove_punctuations(Sentence.lower())) 
    
  filtered_sentence = [] 
  
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 
  return filtered_sentence  

In [0]:
twitter=pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")
df = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz",lines=True)

In [0]:
def remove_punctuations(data):
  data = data
  translator = str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890')
  data = data.translate(translator)
  return data

In [7]:
twitter = twitter.head(1000)
Sentence = twitter['TweetText'][2]
Sentence

"Hilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY"

In [8]:
sent = stop_tokenize(Sentence)
sent

['hilarious',
 'youtube',
 'video',
 'guy',
 'duet',
 'apple',
 'siri',
 'pretty',
 'much',
 'sums',
 'love',
 'affair',
 'httptcoexbnqjy']

In [0]:
def convert(lst):
  lst = stop_tokenize(lst)        
  return ' '.join(lst)

In [0]:
#Remove non english words
def remove_non_english(Sentence):

  words = set(nltk.corpus.words.words())

  sent = Sentence
  Clean = " ".join(w for w in nltk.wordpunct_tokenize(sent) \
            if w.lower() in words or not w.isalpha())
  return Clean

In [0]:
twitter['Clean_tweet'] = twitter['TweetText'].apply(convert)
twitter['Clean_tweet'] = twitter['Clean_tweet'].apply(remove_punctuations)
twitter['Clean_tweet'] = twitter['Clean_tweet'].apply(remove_non_english)

In [13]:
twitter['Clean_tweet']

0                                        apple get crack
1                                  apple carrier support
2      hilarious video guy duet apple pretty much lov...
3                      rim made easy switch apple see ya
4                        reason got twitter thanks apple
                             ...                        
995                      apple upgrade helpful maddening
996    hold apple last really like music selection mu...
997                 win apple touch get hello world baby
998                             poor substitute yo apple
999                                       apple scrapple
Name: Clean_tweet, Length: 1000, dtype: object

In [14]:
steps = [('text',CountVectorizer()),('ensemble',RandomForestClassifier())]
pipe = Pipeline(steps)
twitter.columns

Index(['Topic', 'Sentiment', 'TweetId', 'TweetDate', 'TweetText',
       'Clean_tweet'],
      dtype='object')

In [15]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(twitter['Clean_tweet'])
print(vectorizer.get_feature_names())
x_train,x_test,y_train,y_test = tts(X,twitter['Sentiment'],test_size=0.3, random_state=30)

['ability', 'able', 'absolutely', 'accent', 'access', 'accidently', 'account', 'acquisition', 'across', 'active', 'activity', 'actually', 'add', 'added', 'addicted', 'address', 'admit', 'adobe', 'advance', 'advanced', 'advertise', 'advertising', 'advisor', 'affair', 'afford', 'agree', 'ah', 'ai', 'ailing', 'air', 'airdrop', 'airport', 'aka', 'ala', 'alamo', 'alarm', 'album', 'alcohol', 'alert', 'alive', 'allow', 'ally', 'almost', 'alone', 'already', 'also', 'although', 'alto', 'always', 'amazed', 'amazing', 'amongst', 'amount', 'amused', 'analysis', 'and', 'android', 'angry', 'annoying', 'another', 'answer', 'antitrust', 'anyone', 'anything', 'anyway', 'apologize', 'apology', 'apparently', 'appear', 'apple', 'apply', 'appointment', 'appropriately', 'apt', 'architecture', 'arent', 'argument', 'around', 'arrival', 'arrogant', 'art', 'article', 'artist', 'aside', 'ask', 'ass', 'assistant', 'ate', 'attention', 'audio', 'authorization', 'authorize', 'auto', 'available', 'ave', 'average', 'a

In [0]:
x_train = pd.DataFrame(x_train.toarray(),columns=vectorizer.get_feature_names())
x_test = pd.DataFrame(x_test.toarray(),columns=vectorizer.get_feature_names())

In [0]:
##Apply RandomForestClassifier and LightGBM
##write a report on the classification report :Precision,Recall,Accuracy note 
##change cut off and do the same:0.7,0.8,0.6
##And write  note on how precision recall and accuarcy change with the change in threshold

In [0]:
 rf = RandomForestClassifier()

In [0]:
params = {'max_depth': [10, 20, 30, None],
 'max_features': ['sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': range(10,50,10)}
 
gs_rf = GridSearchCV(estimator = rf, param_grid = params, 
                          cv = 5, n_jobs = 10, verbose = 1)

In [20]:
gs_rf.fit(x_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    9.3s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   20.2s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   40.8s
[Parallel(n_jobs=10)]: Done 720 out of 720 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [21]:
print(classification_report(gs_rf.predict(x_test),y_test))

              precision    recall  f1-score   support

    negative       0.57      0.49      0.53       103
     neutral       0.73      0.67      0.70       178
    positive       0.29      0.74      0.42        19

    accuracy                           0.61       300
   macro avg       0.53      0.63      0.55       300
weighted avg       0.65      0.61      0.62       300



In [0]:
##Precision and recall of this model is respectable for neutral but bad on positive and negative meaning the model is able to decently predict if a twee is neutral or not but not able to predict the postivity or the negativity of the tweet.
##Accuracy is bad but to increase the accuracy the model overfits on the training data giving worse accuracy on the test data

In [24]:
!pip install lightgbm
import lightgbm as lgb

In [0]:
params = {
    "objective" : "multiclass",
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':3,
    'learning_rate':0.01,
    'max_depth': 7,
    'num_leaves': 127,
    'feature_fraction': 0.4,
    'bagging_freq': 10,
    'num_iterations':1000 ,
    'max_bin' : 32}


In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()  
d_train = lgb.Dataset(x_train,le.fit_transform(y_train))

In [27]:
clf = lgb.train( params,d_train, 100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [0]:
import numpy as np
preds = clf.predict(x_test)
best_preds= [np.argmax(line) for line in preds]

In [29]:
np.bincount((le.fit_transform(y_train)))

array([229, 355, 116])

In [30]:
print(classification_report(le.inverse_transform(best_preds),y_test))

              precision    recall  f1-score   support

    negative       0.14      0.39      0.20        31
     neutral       0.90      0.57      0.70       258
    positive       0.15      0.64      0.24        11

    accuracy                           0.56       300
   macro avg       0.39      0.53      0.38       300
weighted avg       0.79      0.56      0.63       300



In [0]:
##The precision of neutral class is high meaning our model mostly classifies the data that it captures correctly on being neutral or not but the other 2 are bad since our model cant classify it well even after hyperparameter tuning
##Accuracy is bad but to increase the accuracy the model overfits on the training data giving worse accuracy on the test data

In [33]:
np.bincount((best_preds))

array([ 31, 258,  11])

In [34]:
import numpy as np
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print(classification_report(le.inverse_transform(best_preds_train),y_train))

              precision    recall  f1-score   support

    negative       0.29      0.65      0.40       101
     neutral       0.93      0.57      0.70       582
    positive       0.09      0.65      0.17        17

    accuracy                           0.58       700
   macro avg       0.44      0.62      0.42       700
weighted avg       0.81      0.58      0.65       700



In [35]:
###for lgbm
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_1(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.6:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.6 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_1(preds)),y_train))
      
###For rf
pred_rf = gs_rf.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(gs_rf.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(le.inverse_transform(thresh_1(pred_rf)),y_test) )



Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.29      0.65      0.40       101
     neutral       0.93      0.57      0.70       582
    positive       0.09      0.65      0.17        17

    accuracy                           0.58       700
   macro avg       0.44      0.62      0.42       700
weighted avg       0.81      0.58      0.65       700


 0.6 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.15      0.77      0.25        44
     neutral       0.37      0.73      0.50       181
    positive       0.81      0.20      0.32       475

    accuracy                           0.37       700
   macro avg       0.44      0.57      0.35       700
weighted avg       0.66      0.37      0.36       700

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.57      0.49      0.53       103
     neutral       0.73      0.67      0

In [0]:
##We see that increasing the threshold increases the recall of the negative and neutral classes but reduces the precision across all the classes for rf model
##We see that increasing the threshold increases the recall of the negative and neutral classes but reduces the precision across all the classes for lgbm model

In [38]:
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_2(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.7:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.7 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_2(preds)),y_train))
###For rf
pred_rf = gs_rf.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(gs_rf.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(le.inverse_transform(thresh_2(pred_rf)),y_test) )
      


Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.29      0.65      0.40       101
     neutral       0.93      0.57      0.70       582
    positive       0.09      0.65      0.17        17

    accuracy                           0.58       700
   macro avg       0.44      0.62      0.42       700
weighted avg       0.81      0.58      0.65       700


 0.7 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.09      0.80      0.16        25
     neutral       0.08      0.72      0.14        39
    positive       0.95      0.17      0.29       636

    accuracy                           0.23       700
   macro avg       0.37      0.56      0.20       700
weighted avg       0.87      0.23      0.28       700

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.57      0.49      0.53       103
     neutral       0.73      0.67      0

In [0]:
##We see that increasing the threshold increases the recall of the negative and neutral classes but reduces the precision across all the classes for rf model
##We see that increasing the threshold increases the recall of the negative and neutral classes but reduces the precision across all the classes for lgbm model

In [41]:
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_3(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.8:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.8 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_3(preds)),y_train))
      
###For rf
pred_rf = gs_rf.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(gs_rf.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(le.inverse_transform(thresh_3(pred_rf)),y_test) )


Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.29      0.65      0.40       101
     neutral       0.93      0.57      0.70       582
    positive       0.09      0.65      0.17        17

    accuracy                           0.58       700
   macro avg       0.44      0.62      0.42       700
weighted avg       0.81      0.58      0.65       700


 0.8 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.03      1.00      0.05         6
     neutral       0.03      1.00      0.07        12
    positive       1.00      0.17      0.29       682

    accuracy                           0.19       700
   macro avg       0.35      0.72      0.14       700
weighted avg       0.98      0.19      0.28       700

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.57      0.49      0.53       103
     neutral       0.73      0.67      0

In [0]:
##We see that increasing the threshold increases the recall of the negative and neutral classes but reduces the precision across all the classes for rf model

##We see that increasing the threshold increases the recall of the negative and neutral classes but reduces the precision across all the classes for lgbm model